# Criação do Catálogo e Schemas - Smart Claims Dev

Este notebook cria a estrutura base do projeto no Databricks utilizando **Unity Catalog** (Lakeflow), incluindo:

- Catálogo principal: `smart_claims_dev`
- Schemas para arquitetura Medallion:
  - `00_landing` - Zona de recepção de dados brutos
  - `01_bronze` - Dados brutos preservados imutavelmente
  - `02_silver` - Dados limpos, validados e enriquecidos
  - `03_gold` - Dados agregados e modelados para consumo
  - `default` - Schema padrão
  - `information_schema` - Metadados do sistema

## Nota sobre Execução
Este notebook utiliza **Python** para executar comandos SQL via `spark.sql()`, permitindo melhor controle de fluxo, tratamento de erros e logging. Cada comando SQL está em uma célula separada para facilitar execução e debugging.


## Parte 1: Criar o Catálogo

**COMANDO: CREATE CATALOG**

O que faz:
- Cria um novo catálogo no Unity Catalog (sistema de governança de dados do Databricks)
- Um catálogo é um container de nível superior que agrupa schemas relacionados
- Permite isolamento e organização de dados por projeto/ambiente (dev, prod, etc.)
- O catálogo possui permissões próprias e pode ser compartilhado entre workspaces
- É o nível mais alto na hierarquia: CATALOG > SCHEMA > TABLE

**Documentação oficial:** [Unity Catalog - CREATE CATALOG](https://docs.databricks.com/en/connect/unity-catalog/create-catalog.html)


In [ ]:
# ============================================================
# CRIAR O CATÁLOGO smart_claims_dev
# ============================================================
# Comando SQL: CREATE CATALOG IF NOT EXISTS
# Cria o catálogo principal do projeto com propriedades customizadas
# ============================================================

create_catalog_sql = """
CREATE CATALOG IF NOT EXISTS smart_claims_dev
COMMENT 'Catálogo principal para o projeto Smart Claims - Ambiente de Desenvolvimento'
"""

try:
    spark.sql(create_catalog_sql)
    print("✅ Catálogo 'smart_claims_dev' criado ou já existe")
except Exception as e:
    print(f"❌ Erro ao criar catálogo: {str(e)}")
    raise


In [ ]:
# ============================================================
# USAR O CATÁLOGO CRIADO
# ============================================================
# Comando SQL: USE CATALOG
# Define o catálogo ativo para os comandos subsequentes
# ============================================================

use_catalog_sql = "USE CATALOG smart_claims_dev"

try:
    spark.sql(use_catalog_sql)
    print("✅ Catálogo 'smart_claims_dev' definido como ativo")
except Exception as e:
    print(f"❌ Erro ao usar catálogo: {str(e)}")
    raise


## Parte 2: Criar os Schemas (Databases)

**COMANDO: CREATE SCHEMA**

O que faz:
- Cria um schema (também chamado de database) dentro do catálogo
- Schemas organizam tabelas relacionadas logicamente
- Cada schema pode ter permissões e políticas de retenção próprias
- É o segundo nível na hierarquia: CATALOG > SCHEMA > TABLE

**Documentação oficial:** [Unity Catalog - CREATE SCHEMA](https://docs.databricks.com/en/connect/unity-catalog/create-schema.html)


In [ ]:
# ============================================================
# CRIAR SCHEMA 00_landing
# ============================================================
# Schema para zona de recepção de dados brutos
# Normalmente recebe dados diretamente de sistemas externos
# Retenção: 7 dias (temporário)
# ============================================================

create_landing_schema_sql = """
CREATE SCHEMA IF NOT EXISTS smart_claims_dev.`00_landing`
COMMENT 'Zona de landing - recepção de dados brutos de sistemas externos'
"""

try:
    spark.sql(create_landing_schema_sql)
    print("✅ Schema '00_landing' criado ou já existe")
except Exception as e:
    print(f"❌ Erro ao criar schema 00_landing: {str(e)}")
    raise


In [ ]:
# ============================================================
# CRIAR SCHEMA 01_bronze
# ============================================================
# Camada Bronze: Preservação imutável dos dados brutos
# Mantém uma cópia permanente dos dados originais (Data Lake Pattern)
# Retenção: 365 dias
# ============================================================

create_bronze_schema_sql = """
CREATE SCHEMA IF NOT EXISTS smart_claims_dev.`01_bronze`
COMMENT 'Camada Bronze - dados brutos preservados de forma imutável'
"""

try:
    spark.sql(create_bronze_schema_sql)
    print("✅ Schema '01_bronze' criado ou já existe")
except Exception as e:
    print(f"❌ Erro ao criar schema 01_bronze: {str(e)}")
    raise


In [ ]:
# ============================================================
# CRIAR SCHEMA 02_silver
# ============================================================
# Camada Silver: Dados limpos, validados e enriquecidos
# Dados transformados com qualidade garantida
# Retenção: 730 dias
# ============================================================

create_silver_schema_sql = """
CREATE SCHEMA IF NOT EXISTS smart_claims_dev.`02_silver`
COMMENT 'Camada Silver - dados limpos, validados e enriquecidos'
"""

try:
    spark.sql(create_silver_schema_sql)
    print("✅ Schema '02_silver' criado ou já existe")
except Exception as e:
    print(f"❌ Erro ao criar schema 02_silver: {str(e)}")
    raise


In [ ]:
# ============================================================
# CRIAR SCHEMA 03_gold
# ============================================================
# Camada Gold: Dados agregados e modelados para consumo final
# Dados otimizados para dashboards, relatórios e ML
# Retenção: 2555 dias (~7 anos)
# ============================================================

create_gold_schema_sql = """
CREATE SCHEMA IF NOT EXISTS smart_claims_dev.`03_gold`
COMMENT 'Camada Gold - dados agregados e modelados para consumo final'
"""

try:
    spark.sql(create_gold_schema_sql)
    print("✅ Schema '03_gold' criado ou já existe")
except Exception as e:
    print(f"❌ Erro ao criar schema 03_gold: {str(e)}")
    raise


In [ ]:
# ============================================================
# CRIAR SCHEMA default
# ============================================================
# Schema padrão do catálogo
# Utilizado para objetos que não precisam de organização específica
# ============================================================

create_default_schema_sql = """
CREATE SCHEMA IF NOT EXISTS smart_claims_dev.default
COMMENT 'Schema padrão do catálogo smart_claims_dev'
"""

try:
    spark.sql(create_default_schema_sql)
    print("✅ Schema 'default' criado ou já existe")
except Exception as e:
    print(f"❌ Erro ao criar schema default: {str(e)}")
    raise


## Parte 3: Verificação e Consulta dos Objetos Criados

Agora vamos verificar se todos os objetos foram criados com sucesso usando comandos de consulta.


In [ ]:
# ============================================================
# VERIFICAR CATÁLOGOS CRIADOS
# ============================================================
# Comando SQL: SHOW CATALOGS
# Lista todos os catálogos visíveis para o usuário atual
# ============================================================

show_catalogs_sql = "SHOW CATALOGS LIKE 'smart_claims*'"

try:
    catalogs_df = spark.sql(show_catalogs_sql)
    print("📋 Catálogos encontrados:")
    catalogs_df.show(truncate=False)
except Exception as e:
    print(f"❌ Erro ao listar catálogos: {str(e)}")


In [ ]:
# ============================================================
# VERIFICAR SCHEMAS NO CATÁLOGO
# ============================================================
# Comando SQL: SHOW SCHEMAS
# Lista todos os schemas dentro do catálogo atual
# ============================================================

show_schemas_sql = "SHOW SCHEMAS IN CATALOG smart_claims_dev"

try:
    schemas_df = spark.sql(show_schemas_sql)
    print("📋 Schemas encontrados no catálogo 'smart_claims_dev':")
    schemas_df.show(truncate=False)
except Exception as e:
    print(f"❌ Erro ao listar schemas: {str(e)}")


In [ ]:
# ============================================================
# DESCREVER O CATÁLOGO
# ============================================================
# Comando SQL: DESCRIBE CATALOG
# Mostra informações detalhadas sobre o catálogo
# ============================================================

describe_catalog_sql = "DESCRIBE CATALOG smart_claims_dev"

try:
    catalog_info_df = spark.sql(describe_catalog_sql)
    print("📋 Informações do catálogo 'smart_claims_dev':")
    catalog_info_df.show(truncate=False)
except Exception as e:
    print(f"❌ Erro ao descrever catálogo: {str(e)}")


In [ ]:
# ============================================================
# DESCREVER SCHEMA 00_landing
# ============================================================
# Comando SQL: DESCRIBE SCHEMA
# Mostra informações detalhadas sobre o schema
# ============================================================

describe_landing_sql = "DESCRIBE SCHEMA smart_claims_dev.`00_landing`"

try:
    landing_info_df = spark.sql(describe_landing_sql)
    print("📋 Informações do schema '00_landing':")
    landing_info_df.show(truncate=False)
except Exception as e:
    print(f"❌ Erro ao descrever schema 00_landing: {str(e)}")


In [ ]:
# ============================================================
# DESCREVER SCHEMA 01_bronze
# ============================================================

describe_bronze_sql = "DESCRIBE SCHEMA smart_claims_dev.`01_bronze`"

try:
    bronze_info_df = spark.sql(describe_bronze_sql)
    print("📋 Informações do schema '01_bronze':")
    bronze_info_df.show(truncate=False)
except Exception as e:
    print(f"❌ Erro ao descrever schema 01_bronze: {str(e)}")


In [ ]:
# ============================================================
# DESCREVER SCHEMA 02_silver
# ============================================================

describe_silver_sql = "DESCRIBE SCHEMA smart_claims_dev.`02_silver`"

try:
    silver_info_df = spark.sql(describe_silver_sql)
    print("📋 Informações do schema '02_silver':")
    silver_info_df.show(truncate=False)
except Exception as e:
    print(f"❌ Erro ao descrever schema 02_silver: {str(e)}")


In [ ]:
# ============================================================
# DESCREVER SCHEMA 03_gold
# ============================================================

describe_gold_sql = "DESCRIBE SCHEMA smart_claims_dev.`03_gold`"

try:
    gold_info_df = spark.sql(describe_gold_sql)
    print("📋 Informações do schema '03_gold':")
    gold_info_df.show(truncate=False)
except Exception as e:
    print(f"❌ Erro ao descrever schema 03_gold: {str(e)}")


## Parte 4: Resumo e Validação Final

Vamos fazer uma validação final da estrutura criada.


In [ ]:
# ============================================================
# RESUMO DA ESTRUTURA CRIADA
# ============================================================
# Validação final: contar schemas criados
# ============================================================

try:
    # Listar todos os schemas
    schemas_df = spark.sql("SHOW SCHEMAS IN CATALOG smart_claims_dev")
    schema_count = schemas_df.count()
    
    print("=" * 60)
    print("✅ RESUMO DA ESTRUTURA CRIADA")
    print("=" * 60)
    print(f"\n📦 Catálogo: smart_claims_dev")
    print(f"📁 Total de Schemas: {schema_count}")
    print("\n📋 Schemas criados:")
    
    # Listar nomes dos schemas
    schemas_list = [row['databaseName'] for row in schemas_df.collect()]
    for schema in sorted(schemas_list):
        print(f"   - {schema}")
    
    print("\n" + "=" * 60)
    print("✅ Estrutura criada com sucesso!")
    print("=" * 60)
    
except Exception as e:
    print(f"❌ Erro ao gerar resumo: {str(e)}")


## Estrutura Final Criada

```
CATALOG: smart_claims_dev
  ├── SCHEMA: 00_landing      (Zona de recepção)
  ├── SCHEMA: 01_bronze        (Dados brutos imutáveis)
  ├── SCHEMA: 02_silver        (Dados limpos e curados)
  ├── SCHEMA: 03_gold          (Dados agregados e modelados)
  ├── SCHEMA: default          (Schema padrão)
  └── SCHEMA: information_schema (Metadados do sistema - automático)
```

✅ **Notebook concluído com sucesso!**

Todos os objetos foram criados e estão prontos para uso. A estrutura segue o padrão **Medallion Architecture** (Lakehouse) para organização de dados em camadas.

### Próximos Passos

Após concluir a Task_001, você pode:

- **Task_002**: Criar volumes no schema `00_landing` e carregar arquivos CSV
- **Task_003**: Criar tabelas Delta na camada `01_bronze` a partir dos arquivos
- **Task_004**: Implementar transformações (Bronze → Silver → Gold)

### Documentação de Referência

- [Unity Catalog - CREATE CATALOG](https://docs.databricks.com/en/connect/unity-catalog/create-catalog.html)
- [Unity Catalog - CREATE SCHEMA](https://docs.databricks.com/en/connect/unity-catalog/create-schema.html)
- [Unity Catalog - SHOW CATALOGS](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-show-catalogs.html)
- [Medallion Architecture](https://www.databricks.com/glossary/medallion-architecture)


Eu quero 

# Criação do Catálogo e Schemas - Smart Claims Dev

Este notebook cria a estrutura base do projeto no Databricks utilizando **Unity Catalog** (Lakeflow), incluindo:

- Catálogo principal: `smart_claims_dev`
- Schemas para arquitetura Medallion:
  - `00_landing` - Zona de recepção de dados brutos
  - `01_bronze` - Dados brutos preservados imutavelmente
  - `02_silver` - Dados limpos, validados e enriquecidos
  - `03_gold` - Dados agregados e modelados para consumo
  - `default` - Schema padrão
  - `information_schema` - Metadados do sistema



## Parte 1: Criar o Catálogo

**COMANDO: CREATE CATALOG**

O que faz:
- Cria um novo catálogo no Unity Catalog (sistema de governança de dados do Databricks)
- Um catálogo é um container de nível superior que agrupa schemas relacionados
- Permite isolamento e organização de dados por projeto/ambiente (dev, prod, etc.)
- O catálogo possui permissões próprias e pode ser compartilhado entre workspaces
- É o nível mais alto na hierarquia: CATALOG > SCHEMA > TABLE



In [0]:
%sql
-- Create the catalog smart_claims_dev
CREATE CATALOG IF NOT EXISTS smart_claims_dev
COMMENT 'Catálogo principal para o projeto Smart Claims - Ambiente de Desenvolvimento';

-- Use the created catalog
USE CATALOG smart_claims_dev;

## Parte 2: Criar os Schemas (Databases)

**COMANDO: CREATE SCHEMA**

O que faz:
- Cria um schema (também chamado de database) dentro do catálogo
- Schemas organizam tabelas relacionadas logicamente
- Cada schema pode ter permissões e políticas de retenção próprias
- É o segundo nível na hierarquia: CATALOG > SCHEMA > TABLE



In [0]:
%sql
-- Criando catálogo (se ainda não existir)
CREATE CATALOG IF NOT EXISTS smart_claims_dev
COMMENT 'Catálogo de desenvolvimento Smart Claims';

USE CATALOG smart_claims_dev;

-- Criar schemas/layers
CREATE SCHEMA IF NOT EXISTS `00_landing`
COMMENT 'Landing zone - dados brutos temporários';

CREATE SCHEMA IF NOT EXISTS `01_bronze`
COMMENT 'Bronze - dados ingeridos e padronizados';

CREATE SCHEMA IF NOT EXISTS `02_silver`
COMMENT 'Silver - dados tratados e refinados';

CREATE SCHEMA IF NOT EXISTS `03_gold`
COMMENT 'Gold - dados analíticos e métricas';

-- Default já existe mas vamos garantir
CREATE SCHEMA IF NOT EXISTS `default`
COMMENT 'Schema padrão para objetos auxiliares';


## Parte 3: Verificação e Consulta dos Objetos Criados



`SHOW CATALOGS` O que faz:
- Lista todos os catálogos visíveis para o usuário atual
- Mostra metadados básicos dos catálogos
- Útil para verificar permissões e catálogos disponíveis


In [0]:
%sql
SHOW CATALOGS LIKE 'smart_claims*';

In [0]:
-- COMANDO: SHOW SCHEMAS
-- O que faz:
-- - Lista todos os schemas dentro do catálogo atual
-- - Mostra nome, tipo (MANAGED/EXTERNAL), localização, etc.
-- - Útil para verificar a estrutura criada

SHOW SCHEMAS IN CATALOG smart_claims_dev;



In [0]:
-- COMANDO: DESCRIBE CATALOG
-- O que faz:
-- - Mostra informações detalhadas sobre um catálogo específico
-- - Exibe propriedades, comentários, e metadados
-- - Útil para documentação e auditoria

DESCRIBE CATALOG smart_claims_dev;



In [0]:
-- COMANDO: DESCRIBE SCHEMA
-- O que faz:
-- - Mostra informações detalhadas sobre um schema específico
-- - Exibe propriedades, comentários, localização, e metadados
-- - Útil para entender a configuração de cada camada

DESCRIBE SCHEMA smart_claims_dev.`00_landing`;
DESCRIBE SCHEMA smart_claims_dev.`01_bronze`;
DESCRIBE SCHEMA smart_claims_dev.`02_silver`;
DESCRIBE SCHEMA smart_claims_dev.`03_gold`;



## Parte 4: Configurações Adicionais (Opcional)



In [0]:
-- COMANDO: ALTER CATALOG
-- O que faz:
-- - Modifica propriedades de um catálogo existente
-- - Permite atualizar comentários, propriedades, etc.
-- - Útil para manutenção e atualização de metadados

-- Exemplo de atualização de comentário (opcional)
-- ALTER CATALOG smart_claims_dev SET COMMENT 'Catálogo atualizado em ' || current_timestamp();



In [0]:
-- COMANDO: GRANT/REVOKE
-- O que faz:
-- - Gerencia permissões em catálogos e schemas
-- - Permite controle de acesso granular
-- - Essencial para segurança e governança

-- Exemplo de permissões (ajustar conforme necessário):
-- GRANT USE CATALOG ON CATALOG smart_claims_dev TO `data_engineers`;
-- GRANT CREATE SCHEMA ON CATALOG smart_claims_dev TO `data_engineers`;
-- GRANT ALL PRIVILEGES ON SCHEMA smart_claims_dev.`00_landing` TO `data_engineers`;



## Resumo da Estrutura Criada

```
CATALOG: smart_claims_dev
  ├── SCHEMA: 00_landing      (Zona de recepção)
  ├── SCHEMA: 01_bronze        (Dados brutos imutáveis)
  ├── SCHEMA: 02_silver        (Dados limpos e curados)
  ├── SCHEMA: 03_gold          (Dados agregados e modelados)
  ├── SCHEMA: default          (Schema padrão)
  └── SCHEMA: information_schema (Metadados do sistema)
```

✅ **Notebook concluído com sucesso!**

Todos os objetos foram criados e estão prontos para uso. Execute as células sequencialmente para criar a estrutura completa.

